<a href="https://colab.research.google.com/github/OlanreleOladeji/Demand-forecast--XGBoost/blob/main/Products_quantity_forecasting_model_(XGB).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data upload and preprocessing

In [225]:
import pandas as pd

In [226]:
import numpy as np

# forecast monthly births with xgboost
from numpy import asarray

In [227]:
from google.colab import files

In [228]:
uploaded = files.upload()

Saving 1pharma-data.csv to 1pharma-data (3).csv


In [ ]:
df = pd.read_csv('1pharma-data.csv')

In [229]:
df.head()

,Year,Month,Product_ID,Sales,Revenue
0,2017,April,Abatatriptan,1410.0,1046220.0
1,2017,April,Abilovir Aprotasol,1974.0,531006.0
2,2017,April,Abobozolid,1350.0,101250.0
3,2017,April,Abranatal Lysoprosate,3767.0,2565327.0
4,2017,April,Abtasol,594.0,447876.0


Check dataset structure

In [230]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11520 entries, 0 to 11519
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Year        11520 non-null  int64  
 1   Month       11520 non-null  object 
 2   Product_ID  11520 non-null  object 
 3   Sales       11520 non-null  float64
 4   Revenue     11520 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 450.1+ KB


Data Exploration and Cleaning

Print the column headings

In [231]:
print(df.columns.tolist())


['Year', 'Month', 'Product_ID', 'Sales', 'Revenue']


Dropping columns

In [233]:
df= df.drop(['Channel', 'Sub-channel', 'Product Class', 'Name of Sales Rep', 'Price','Manager', 'Sales Team', 'Customer Name', 'City', 'Country', 'Distributor', 'Latitude', 'Longitude', 'Channel', 'Sub-channel', 'Product Class', 'Unnamed: 18'], axis=1)

KeyError: "['Channel', 'Sub-channel', 'Product Class', 'Name of Sales Rep', 'Price', 'Manager', 'Sales Team', 'Customer Name', 'City', 'Country', 'Distributor', 'Latitude', 'Longitude', 'Channel', 'Sub-channel', 'Product Class', 'Unnamed: 18'] not found in axis"

In [232]:
df.head()

,Year,Month,Product_ID,Sales,Revenue
0,2017,April,Abatatriptan,1410.0,1046220.0
1,2017,April,Abilovir Aprotasol,1974.0,531006.0
2,2017,April,Abobozolid,1350.0,101250.0
3,2017,April,Abranatal Lysoprosate,3767.0,2565327.0
4,2017,April,Abtasol,594.0,447876.0


Sum the number of items in each month per year

In [ ]:
monthly_sales = df.groupby(['Year', 'Month', 'Product_ID'])['Sales'].sum().reset_index()

In [ ]:
Monthly_Revenue= df.groupby(['Year', 'Month', 'Product_ID'])['Revenue'].sum().reset_index()

In [ ]:
merged_df = pd.merge(monthly_sales, Monthly_Revenue, on=['Year', 'Month', 'Product_ID'])

In [ ]:
merged_df.head()

In [ ]:
df= merged_df

In [ ]:
df

In [ ]:
#print(df['Product_ID'].unique())

Feature Engineering

Split Dataset

In [ ]:
from sklearn.model_selection import train_test_split
X = df[['Revenue']]
y = df[['Sales']]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X

In [ ]:
y

Dataset Scaling using MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

Scaling the train_X and test_X :

In [ ]:
scaler_X = MinMaxScaler()

In [234]:
scaler_X.fit(X)

MinMaxScaler()

In [235]:
train_X_Scaled = scaler_X.transform(train_X)

In [236]:
train_X_Scaled = pd.DataFrame(train_X_Scaled, columns=['Revenue'])

In [237]:
train_X_Scaled.head(10)

,Revenue
0,0.049249
1,0.048355
2,0.070333
3,0.058062
4,0.049088
5,0.051306
6,0.069561
7,0.052523
8,0.047038
9,0.053201


In [238]:
test_X_Scaled = scaler_X.transform(test_X)

In [239]:
test_X_Scaled = pd.DataFrame(test_X_Scaled, columns=['Revenue'])

In [240]:
test_X_Scaled.head(10)

,Revenue
0,0.048843
1,0.055124
2,0.052252
3,0.081784
4,0.055866
5,0.051466
6,0.048486
7,0.072890
8,0.052004
9,0.068085


Scaling the train_y and test_y :

In [241]:
scaler_y = MinMaxScaler()

In [242]:
scaler_y.fit(y)

MinMaxScaler()

In [243]:
train_y_Scaled = scaler_y.transform(train_y)

In [244]:
train_y_Scaled = pd.DataFrame(train_y_Scaled, columns=['Sales'])

In [245]:
train_y_Scaled.head(10)

,Sales
0,0.061046
1,0.062750
2,0.070812
3,0.061553
4,0.059503
5,0.056642
6,0.076824
7,0.055990
8,0.053812
9,0.061569


In [246]:
test_y_Scaled = scaler_y.transform(test_y)

In [247]:
test_y_Scaled = pd.DataFrame(test_y_Scaled, columns=['Sales'])

In [248]:
test_y_Scaled.head(10)

,Sales
0,0.053989
1,0.058418
2,0.056537
3,0.093880
4,0.069936
5,0.057445
6,0.056199
7,0.091831
8,0.055291
9,0.070490


XGBoost Algorithm

In [249]:
from xgboost import XGBRegressor

In [250]:
model = XGBRegressor()

In [251]:
model.fit(train_X_Scaled, train_y_Scaled)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

Make predictions on the test dataset (test_X_Scaled):

In [254]:
y_predict = model.predict(test_X_Scaled)

In [255]:
print(predictions)

[0.0612699  0.06489413 0.06278524 ... 0.06202662 0.06340696 0.06214616]


Summarize progress

In [278]:
Progress_summary_df = pd.DataFrame({
        'Actual_Scaled_Sales': test_y_Scaled['Sales'],
        'Predictions': y_predict[:test_y_Scaled.shape[0]]
    })

print("Progress summary:")

display(Progress_summary_df.head(5))

Progress summary:


,Actual_Scaled_Sales,Predictions
0,0.053989,0.061270
1,0.058418,0.064894
2,0.056537,0.062785
3,0.093880,0.089145
4,0.069936,0.065230


In [ ]:
from matplotlib import pyplot
pyplot.plot(y, label='Expected')
pyplot.plot(yhat, label='Predicted')
pyplot.legend()
pyplot.show()

Evaluate model with performance metrics

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error

In [ ]:
mean_squared_error(test_y_Scaled, predictions)

0.00038125834544189274

In [ ]:
mean_absolute_error(test_y_Scaled, predictions)

0.008018013089895248

In [ ]:
 root_mean_squared_error(test_y_Scaled, predictions)

0.019525837153196335

In [ ]:
#pip install m5-wrmsse

In [ ]:
#from m5_wrmsse import wrmsse

In [ ]:
#y_predict = np.ones((30490,28))

In [ ]:
#wrmsse_score = wrmsse(y_predict)

In [ ]:
#print(wrmsse_score)

Hyperparemeter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    "n_estimators": [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    "learning_rate": [0.01, 0.1],
    "max_depth": [3, 7],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 0.1],
   }

In [ ]:
grid_search = GridSearchCV(estimator= XGBRegressor(), param_grid= param_grid, cv=5, verbose=1)

In [ ]:
grid_search.fit(train_X_Scaled, train_y_Scaled)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 0.1],
                         'learning_rate': [0.01, 0.1], 'max_depth': [3, 7],
                         'n_estimators': [100, 200, 300, 400, 500, 600, 700,
                                          800, 900, 1000],
                         'subsample': [0.8, 1.0]},
             verbose=1)

Best parameters

In [ ]:
grid_search.best_params_

{'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 100,
 'subsample': 0.8}

Make predictions on the test dataset (test_X_Scaled) using grid search best estimator

In [ ]:
predictions_tuned = y_predict_tuned = grid_search.best_estimator_.predict(test_X_Scaled)

In [ ]:
print(predictions_tuned)

[0.06137752 0.06510728 0.06286702 ... 0.06199182 0.06261616 0.06199182]


In [ ]:
mean_squared_error(test_y_Scaled, predictions_tuned)

0.00038102251710370183

In [ ]:
mean_absolute_error(test_y_Scaled, predictions_tuned)

0.007939578965306282

In [ ]:
 root_mean_squared_error(test_y_Scaled, predictions_tuned)

0.019519798457622528

In [ ]:
#from m5_wrmsse import wrmsse

In [ ]:
#y_pred_tuned = np.ones((30490,28))

In [ ]:
#wrmsse_score = wrmsse(y_pred_tuned)
#print(wrmsse_score)